### Visualization for self-reports ###

In [1]:
# IMPORTING MODULES
import glob
import importlib
import matplotlib.pyplot as plt
import numpy as np
import os
import pandas as pd
import scipy.signal as ss
import sys
module_path = os.path.abspath(os.path.join('..', '..', 'src'))
sys.path.append(module_path)
import tools.data_reader_apd as dr_a
import tools.data_reader_wesad as dr_w
import tools.data_reader_popane as dr_p
import tools.display_tools as dt
import train

c:\Users\zhoux\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## APD 

In [117]:
# APD
importlib.reload(dr_a)
importlib.reload(train)

threshold = "fixed"
participant_file = os.path.join(dr_a.Paths.DATA_DIR, "participants_details.csv")
df = pd.read_csv(participant_file)
labels = [
    "Baseline_SUDS", "BugBox_Relax_SUDS", "BugBox_Preparation_SUDS", "BugBox_Exposure_SUDS", "BugBox_Exposure_SUDS", 
    "Speech_Relax_SUDS", "Speech_SUDS", "Speech_Exposure_SUDS", "Speech_Break_SUDS"
]
suds_labels = df.loc[:, ["Participant"] + labels]
mean_suds = np.mean(suds_labels.loc[:, labels])

ha_suds_df = suds_labels.loc[suds_labels['Participant'].isin(dr_a.Groups.ha_participant_indices)]
la_suds_df = suds_labels.loc[suds_labels['Participant'].isin(dr_a.Groups.la_participant_indices)]

In a future version, DataFrame.mean(axis=None) will return a scalar mean over the entire DataFrame. To retain the old behavior, use 'frame.mean(axis=0)' or just 'frame.mean()'


In [ ]:
fig, axs = plt.subplots(7, 4, sharex=True, sharey=True)
fig.set_size_inches(12, 12)
idx = 0
for i in range(7):
    for j in range(4):
        try:
            axs[i][j].hist(ha_suds_df[labels].iloc[idx], bins=list(range(0, 100, 10)))
            axs[i][j].set_title(ha_suds_df["Participant"].iloc[idx], fontdict={"fontsize": 8})
        except Exception as e:
            continue
        idx += 1
print("APD self-reports -- high trait anxiety group")
fig.show()

fig2, axs2 = plt.subplots(7, 4, sharex=True, sharey=True)
fig2.set_size_inches(12, 12)
idx = 0
for i in range(7):
    for j in range(4):
        try:
            axs2[i][j].hist(la_suds_df[labels].iloc[idx], bins=list(range(0, 100, 10)))
            axs2[i][j].set_title(la_suds_df["Participant"].iloc[idx], fontdict={"fontsize": 8})
        except Exception as e:
            continue
        idx += 1
print("APD self-reports -- low trait anxiety group")
fig2.show()

## WESAD

In [ ]:
# WESAD
importlib.reload(dr_w)
importlib.reload(train)

stai_labels = dr_w.get_stai_scores()
labels = [
    "Base_STAI", "TSST_STAI", "Medi_1_STAI", "Fun_STAI", "Medi_2_STAI"
]
fig, axs = plt.subplots(5, 3, sharex=True, sharey=True)
fig.set_size_inches(12, 12)
idx = 0
for i in range(7):
    for j in range(4):
        try:
            axs[i][j].hist(stai_labels[labels].iloc[idx], bins=list(range(20, 80, 10)))
            axs[i][j].set_title(stai_labels["subject"].iloc[idx], fontdict={"fontsize": 8})
        except Exception as e:
            continue
        idx += 1
print("WESAD self-reports")
fig.show()

## POPANE

In [ ]:
# POPANE
importlib.reload(dr_p)
importlib.reload(train)

study = "Study7"
self_report_df = pd.read_csv(os.path.join(dr_p.Paths.METRICS, study, "self_reports.csv"), index_col=0)
labels = self_report_df.columns[1:]
fig, axs = plt.subplots(5, 3, sharex=True, sharey=True)
fig.set_size_inches(9, 9)
idx = 0
for i in range(7):
    for j in range(4):
        try:
            axs[i][j].hist(self_report_df[labels].iloc[idx], bins=list(range(0, 10, 1)))
            axs[i][j].set_title(self_report_df["subject"].iloc[idx], fontdict={"fontsize": 8})
            axs[i][j].set_yticks(list(range(0, 7, 2)))
        except Exception as e:
            continue
        idx += 1
print(f"POPANE Study {study[-1]} self-reports")
fig.show()

## Visualizing labels

In [150]:
importlib.reload(train)
importlib.reload(dr_a)
importlib.reload(dr_w)
importlib.reload(dr_p)

apd_suds_labels = dr_a.get_suds_labels(threshold=threshold)
wesad_stai_labels = dr_w.get_stai_labels(threshold=threshold)
popane_affect_labels = {
    "Study1": [],
    "Study2": [],
    "Study3": [],
    "Study4": [],
    "Study5": [],
    "Study6": [],
    "Study7": []
}

popane_phases = {
    "Study1": dr_p.Study1.ALL,
    "Study2": dr_p.Study2.ALL,
    "Study3": dr_p.Study3.ALL,
    "Study4": dr_p.Study4.ALL,
    "Study5": dr_p.Study5.ALL,
    "Study6": dr_p.Study6.ALL,
    "Study7": dr_p.Study7.ALL
}

for study in popane_affect_labels.keys():
    popane_affect_labels[study] = dr_p.get_affect_labels(study, popane_phases[study], threshold=threshold)

In [183]:
importlib.reload(train)

metrics = [
    train.Metrics.BPM, 
    train.Metrics.RMSSD, 
    train.Metrics.HF_RR, 
    train.Metrics.LF_RR, 
    train.Metrics.IBI, 
    train.Metrics.SDNN, 
    train.Metrics.MEAN_SCL, 
    train.Metrics.SCR_RATE
]

study = "Study5"
x, y = train.Train_POPANE.get_popane_data(study, metrics, popane_phases[study])